In [1]:
#!pip install datasets

In [2]:
import torch

In [3]:
MODEL = "bert-base-uncased"
BATCH_SIZE = 4

In [4]:
# from datasets import load_dataset

# imdb = load_dataset("imdb")

Reusing dataset imdb (C:\Users\fahad\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
from datasets import load_dataset
# sarc_train = load_dataset('csv', data_files=['train-balanced.csv'], delimiter=",", split="train",on_bad_lines='skip')
# print(sarc_train[0])

data_files = {"train": "train-balanced.csv", "test": "test-balanced.csv"}
dataset = load_dataset('csv',data_files=data_files, on_bad_lines='skip')

Using custom data configuration default-7baf4ba717b4988b


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Couldn't cast
0	Your mom	Ronan3463	teenagers	1	-1	-1	2017-04	1491694828	What are you doing 3 hours from now?: string
__index_level_0__: string
__index_level_1__: string
__index_level_2__: string
__index_level_3__: string
__index_level_4__: string
__index_level_5__: string
__index_level_6__: string
__index_level_7__: string
__index_level_8__: string
__index_level_9__: string
__index_level_10__: string
-- schema metadata --
pandas: '{"index_columns": ["__index_level_0__", "__index_level_1__", "__' + 2039
to
{'0\tDang dog': Value(dtype='string', id=None), ' thanks\tMattys1174\tNHLHUT\t1\t-1\t-1\t2017-03\t1490927299\tGreat guess! Congrats!': Value(dtype='string', id=None)}
because column names don't match

In [22]:
sarc_train[0]

{'0\tDang dog': '0\tto summon the powers of the flying spaghetti monster to turn him into money while outside.\tyugiohhero\tCringeAnarchy\t1\t-1\t-1\t2017-02\t1486349313\tcan anynone explain what cash me outside means?',
 ' thanks\tMattys1174\tNHLHUT\t1\t-1\t-1\t2017-03\t1490927299\tGreat guess! Congrats!': None}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\fahad\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-9cc81d8c89dda1a0.arrow
Loading cached processed dataset at C:\Users\fahad\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-8bc51db61ad9f6fd.arrow
Loading cached processed dataset at C:\Users\fahad\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-bf107de2aeeaae5f.arrow


In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments, Trainer
from os import path

output_dir_path = path.join("results")
training_args = TrainingArguments(
    output_dir = output_dir_path,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\fahad\AppData\Roaming\Python\Python37\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 31250


Step,Training Loss
500,0.457300
1000,0.410800
1500,0.428300
2000,0.399000


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json
Saving model checkpoint to results\checkpoint-1000
Configuration saved in results\checkpoint-1000\config.json
Model weights saved in results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in results\checkpoint-1000\special_tokens_map.json
Saving model checkpoint to results\checkpoint-1500
Configuration saved in results\checkpoint-1500\config.json
Model weights saved in results\checkpoint-1500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1500\tokenizer_config.json
Special tokens file saved in results\checkpoint-1500\special_tokens_map.json
Saving model check